In [9]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Load Dataset
DATASET_PATH = r"C:\Users\bhumi\Downloads\dataset_skin disease"  # Change this to your dataset path
IMG_SIZE = 128
NUM_CLASSES = 7  # Change based on the number of skin disease categories

# Function to Load Images and Labels
def load_data(dataset_path, img_size):
    images, labels = [], []
    class_names = os.listdir(dataset_path)  # Folder names as class labels
    
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(dataset_path, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (img_size, img_size))
            images.append(img)
            labels.append(label)
    
    images = np.array(images) / 255.0  # Normalize images
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=len(class_names))  # One-hot encoding
    return images, labels, class_names  # ✅ Corrected indentation

# Load the Data
X, y, class_names = load_data(DATASET_PATH, IMG_SIZE)

# Split Data into Train & Test Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert y_train and y_test to categorical after splitting
y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")


# Data Augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
datagen.fit(X_train)

# CNN Model
def create_cnn_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # ✅ Corrected indentation
    return model

# Build and Train Model
model = create_cnn_model((IMG_SIZE, IMG_SIZE, 3), NUM_CLASSES)

history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_test, y_test))

# Evaluate Model
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

# Save Model
model.save("skin_disease_cnn.h5")

# Prediction Functi


y_train shape: (303, 7), y_test shape: (76, 7)


C:\Users\bhumi\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\bhumi\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 853ms/step - accuracy: 0.7734 - loss: 0.5981 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 739ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 730ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 761ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 729ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 735ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 749ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 731ms/step

Test Accuracy: 1.0000


In [5]:
def display_prediction(img_path, model, class_names, img_size=128):
    predicted_class, prediction_probs = predict_image(img_path, model, class_names, img_size)
    
    # Load the image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for matplotlib
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Predicted: {predicted_class}")
    plt.show()

    print(f"Predicted Class: {predicted_class}")
    print(f"Prediction Probabilities: {prediction_probs}")


In [3]:
test_img_path = ""  # Change this to your test image path
display_prediction(test_img_path, model, class_names)


NameError: name 'model' is not defined